In [14]:
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences 

In [15]:
tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index

X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_train_padded = pad_sequences(X_train_sequences, padding='post', truncating='post', maxlen=40)

X_val_sequences = tokenizer.texts_to_sequences(X_val)
X_val_padded = pad_sequences(X_val_sequences, padding='post', truncating='post', maxlen=40)

X_test_sequences = tokenizer.texts_to_sequences(X_test)
X_test_padded = pad_sequences(X_test_sequences, padding='post', truncating='post', maxlen=40)

In [16]:
len(word_index)

23870

In [17]:
from tensorflow import keras
from tensorflow.keras import layers

In [18]:
embedding_dim = 64
vocab_size = len(word_index) + 1

model = keras.Sequential([
    layers.Embedding(vocab_size, embedding_dim),
    layers.Bidirectional(layers.LSTM(64, return_sequences=True)),
    layers.Bidirectional(layers.LSTM(32)),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 64)          1527744   
                                                                 
 bidirectional (Bidirectiona  (None, None, 128)        66048     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               41216     
 nal)                                                            
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,639,233
Trainable params: 1,639,233
Non-

In [20]:
from tensorflow.keras import callbacks, metrics

In [21]:
METRICS = [
    metrics.BinaryAccuracy(name='accuracy'),
    metrics.Precision(name='precision'),
    metrics.Recall(name='recall')
]

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-4), 
    loss='binary_crossentropy', 
    metrics=METRICS
)

early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    mode='min',
    min_delta=1e-5,
    patience=5,
    verbose=1,
    restore_best_weights=True
)

reduce_learning_rate = callbacks.ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.5, 
    patience=5, 
    min_lr=1e-6, 
    verbose=1
)

In [22]:
import tensorflow as tf

In [23]:
history = model.fit(
    tf.constant(X_train_padded), 
    tf.constant(y_train), 
    validation_data=(tf.constant(X_val_padded), tf.constant(y_val)),
    batch_size=32,
    epochs=30,
    callbacks=[early_stopping, reduce_learning_rate]
)

Epoch 1/30
656/656 [==============================] - 22s 23ms/step - loss: 0.5003 - accuracy: 0.7480 - precision: 0.7780 - recall: 0.6940 - val_loss: 0.3257 - val_accuracy: 0.8647 - val_precision: 0.8717 - val_recall: 0.8554 - lr: 1.0000e-04
Epoch 2/30
656/656 [==============================] - 14s 21ms/step - loss: 0.2314 - accuracy: 0.9122 - precision: 0.9072 - recall: 0.9182 - val_loss: 0.2734 - val_accuracy: 0.8932 - val_precision: 0.9096 - val_recall: 0.8732 - lr: 1.0000e-04
Epoch 3/30
656/656 [==============================] - 13s 21ms/step - loss: 0.1267 - accuracy: 0.9584 - precision: 0.9592 - recall: 0.9576 - val_loss: 0.2693 - val_accuracy: 0.8974 - val_precision: 0.9096 - val_recall: 0.8825 - lr: 1.0000e-04
Epoch 4/30
656/656 [==============================] - 16s 24ms/step - loss: 0.0694 - accuracy: 0.9800 - precision: 0.9814 - recall: 0.9786 - val_loss: 0.3062 - val_accuracy: 0.8979 - val_precision: 0.9124 - val_recall: 0.8803 - lr: 1.0000e-04
Epoch 5/30
656/656 [========